# Reading in Results

In [10]:
import bestreads.text as text
import pandas as pd
import numpy as np
from tqdm import tqdm

data_dir = './data/processed/'
processed_data_train = pd.read_csv(
    data_dir + 'goodreads_books_train_processed.csv', index_col=0)
processed_data_test = pd.read_csv(
    data_dir + 'goodreads_books_test_processed.csv', index_col=0)
tf_idf_table = pd.read_csv(
    data_dir + 'tf_idf_table_train.csv', index_col=0)

In [2]:
genre_and_votes_test = processed_data_test[['genre_1', 'votes_1']]
cleaned_descriptions_test = processed_data_test['cleaned_descriptions']

In [3]:
tf_idf_table.head()

,Religion,Christian,Westerns,Historical,Animals,Spirituality,Psychology,Science Fiction,Plays,Realistic Fiction,...,Adult Fiction,Travel,Philosophy,Poetry,History,Adventure,Paranormal,Thriller,Mystery,Sequential Art
word,,,,,,,,,,,,,,,,,,,,,
much,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
public,0.000036,0.000018,0.000015,0.000013,0.000012,0.000014,0.000070,0.000010,0.000037,0.000051,...,0.000033,0.000025,0.000074,0.000033,0.000042,0.000014,0.000000,0.000014,0.000012,0.000017
debat,0.000323,0.000000,0.000000,0.000015,0.000000,0.000000,0.000258,0.000012,0.000000,0.000000,...,0.000000,0.000062,0.000566,0.000074,0.000199,0.000000,0.000000,0.000000,0.000007,0.000006
unit,0.000084,0.000029,0.000000,0.000105,0.000056,0.000000,0.000100,0.000148,0.000070,0.000000,...,0.000000,0.000194,0.000027,0.000063,0.000365,0.000100,0.000016,0.000233,0.000064,0.000069
state,0.000098,0.000025,0.000000,0.000067,0.000033,0.000019,0.000206,0.000089,0.000103,0.000000,...,0.000000,0.000142,0.000145,0.000064,0.000316,0.000118,0.000009,0.000151,0.000056,0.000047


# Validation

In [5]:
# Here, I will pull from the top 5 genres, but this could be modified later
n=5
column_names = [f'{stub}_{num+1}' 
                for num in range(n)
                for stub in ['genre', 'score']]
predicted_genres_and_scores = {key:[] for key in column_names}

temp_descriptions = cleaned_descriptions_test.sample(n=100)

for description in tqdm(temp_descriptions):
    result = text.query(description, tf_idf_table).iloc[:n]
    for idx in range(n):
        predicted_genres_and_scores[f'genre_{idx+1}'].append(result.index[idx])
        predicted_genres_and_scores[f'score_{idx+1}'].append(result.iloc[idx])
predicted_genres_and_scores = pd.DataFrame.from_dict(predicted_genres_and_scores)
predicted_genres_and_scores.index = temp_descriptions.index

100%|███████████████████████████████████████████████████████| 100/100 [12:30<00:00,  7.50s/it]


In [11]:
correct = []
for true_genre, predicted_genres in zip(genre_and_votes_test['genre_1'].loc[predicted_genres_and_scores.index],
                                        predicted_genres_and_scores
                                        .filter(like='genre').values):
    if true_genre in predicted_genres:
        correct.append(True)
    elif true_genre is not np.nan:
        correct.append(False)

In [12]:
sum(correct)/len(correct)

0.6133333333333333

In [8]:
genre_and_votes_test.loc[predicted_genres_and_scores.index]

,genre_1,votes_1
index,,
1379,Business,941.0
1381,Nonfiction,348.0
2098,Sequential Art,47.0
1680,Plays,23.0
218,Science Fiction,492.0
...,...,...
4689,Horror,1115.0
1490,NaN,NaN
671,Fiction,96.0


In [36]:
predicted_genres_and_scores

,genre_1,score_1,genre_2,score_2,genre_3,score_3,genre_4,score_4,genre_5,score_5
index,,,,,,,,,,
1181,School Stories,0.211101,Family Law,0.141625,Computers,0.124782,Speculative Fiction,0.116717,Roman,0.115397
2784,Political Science,0.050624,World Of Warcraft,0.048098,Sci Fi Fantasy,0.047973,World War II,0.038920,Medical,0.037007
3296,Mental Health,0.240844,Manga,0.223946,Menage,0.220171,Roman,0.206045,Female Authors,0.204915
